# ➕ Script CLI: Aggiunta File Singolo e Organizzazione in Cartelle

## 🎯 Obiettivo del notebook

Questo script è stato progettato per **inserire un singolo file** all’interno di una struttura di cartelle organizzata, con un’interfaccia a **riga di comando (CLI)**.  
Il file viene analizzato in base alla sua **tipologia** e spostato nella **cartella corretta**, con aggiornamento automatico di un file di **log strutturato** (`recap.csv`).

---

## 🧭 Funzionalità principali

- ✅ Analisi del **tipo MIME** del file (audio, immagine, documento)
- 🗂️ Spostamento del file nella relativa **sottocartella** (`audio/`, `immagini/`, `documenti/`)
- 🔧 Creazione automatica della cartella se non esiste
- 📝 Scrittura (o aggiornamento) del file `recap.csv` con:
  - nome del file
  - tipo MIME
  - dimensione (in byte)
---

## ⚙️ Struttura prevista delle cartelle

Dopo l’esecuzione, la cartella `./files` avrà una struttura come la seguente:




---

## 📤 Esecuzione da terminale

Lo script è progettato per essere eseguito con **argomenti CLI**.  
Esempio da riga di comando:

```bash
python addfile.py esempio.jpg -d ./files


---
## 📦 Librerie utilizzate
- os, shutil – gestione di file e percorsi

- mimetypes – identificazione del tipo MIME

- csv – scrittura del file recap.csv

- argparse – gestione degli argomenti da riga di comando

- datetime – generazione del timestamp per il log

---

## 🔒 Controlli e Gestione errori
- Il file viene verificato prima di essere spostato

- Se la directory non esiste, viene creata

- Se recap.csv non esiste, viene generato

In caso di errori (es. file non esistente), viene stampato un messaggio chiaro all’utente

---

## 👨‍💻 Autore:

## ***Franco De Giorgio***  

📅 **Data di completamento:** Luglio 2025

---

In [1]:
import os           # per navigare nel filesystem: elencare file, controllare esistenza, creare directory, ottenere dimensioni
import shutil       # per spostare o copiare file da una cartella all'altra
import mimetypes    # per determinare il tipo MIME di un file (es. 'image/jpeg', 'audio/mpeg')
import csv          # per leggere e scrivere file CSV (usato per generare il log recap.csv)
import argparse     # per gestire argomenti passati da linea di comando (CLI)
import sys          # per terminare lo script in caso di errori critici (es. sys.exit(1))

In [2]:
def get_mime_type(file_path):
    """
    Restituisce il tipo MIME di un file, ad esempio 'image/png' o 'document/txt'.
    E la dimensione del file in byte
    Usa il modulo 'mimetypes' di Python che associa estensioni a tipi standardizzati.
    Args:
        file_path: percorso completo al file
    Returns:
        mime_type: stringa del tipo MIME (es. 'image/png')
        size: dimensione in byte
    """
    mime_type, _ = mimetypes.guess_type(file_path)
    if mime_type is None:
        mime_type = 'Tipo sconosciuto'
    size = os.path.getsize(file_path)
    return mime_type, size

In [3]:
def classifica_file(mime_type):
    """
    Classifica il file in base al tipo MIME.
    Ritorna 'audio', 'immagini', 'documenti' oppure None se non riconosciuto.
    """
    if mime_type is None:
        return None
    if mime_type.startswith('audio'):
        return 'audio'
    elif mime_type.startswith('image'):
        return 'immagini'
    elif mime_type.startswith('application') or mime_type.startswith('text'):
        return 'documenti'
    else:
        return None  # tipo sconosciuto

In [4]:
def sposta_file(file_path, folder_path, categoria):
    """
    Sposta il file nella sottocartella corrispondente alla categoria ('audio', 'immagini', 'documenti').
    Crea la sottocartella se non esiste.

    Args:
        file_path: percorso completo del file da spostare
        folder_path: percorso della cartella principale (es. './files')
        categoria: stringa con il nome della categoria
    """
    destinazione_cartella = os.path.join(folder_path, categoria)

    # Crazione cartella categoria se non esiste
    if not os.path.exists(destinazione_cartella):
        os.makedirs(destinazione_cartella)

    # Costruzione nuovo percorso file
    nome_file = os.path.basename(file_path)
    nuovo_percorso = os.path.join(destinazione_cartella, nome_file)

    # Sposta il file nella nuova posizione
    shutil.move(file_path, nuovo_percorso)

    # Percorso file
    return nuovo_percorso 

In [5]:
def scrivi_log_csv(file_info, csv_path):
    """
    Scrive una riga nel file CSV con le informazioni sul file spostato.

    Args:
        file_info: dizionario con le chiavi 'Nome', 'Tipo', 'Size'
        csv_path: percorso del file recap.csv
    """
    # Verifica se il file recap.csv esiste già
    file_esiste = os.path.exists(csv_path)

    # Apre il file in modalità append (aggiunta)
    with open(csv_path, mode='a', newline='', encoding='utf-8') as csvfile:
        # Definisce le intestazioni
        intestazioni = ['Nome', 'Tipo', 'Size Byte']
        
        # Crea il writer
        writer = csv.DictWriter(csvfile, fieldnames=intestazioni)

        # Scrive l’intestazione solo se il file non esisteva
        if not file_esiste:
            writer.writeheader()
        
        # Scrive la riga del file corrente
        writer.writerow(file_info)


In [ ]:
def main():
    """
    Funzione principale tramite CLI gestisce l'organizzazione di un singolo file:
    lo classifica, sposta e registra un log nel file recap.csv.

    Args:
        folder_path: percorso alla cartella principale (es. './files')
    """
    # definizione dell'interfaccia CLI
    parser = argparse.ArgumentParser(description="Sposta un file in base alla tipologia all'interno della relativa sotto cartella e crea un file recap.csv per il log",
    epilog="Esempio:\n  python addfile.py cane.jpeg -d ./files",
    formatter_class=argparse.RawTextHelpFormatter)
    parser.add_argument('filename', type=str, help="Nome del file da spostare (esempio: cane.jpeg)")
    parser.add_argument('-d', '--directory', type=str, default=None, help="Percorso cartella dove si trova il file (default: input da tastiera)")
    args = parser.parse_args()

    # Se non specificato da CLI, chiedo input manuale
    if args.directory is None:
        folder_path = input("Inserisci il percorso della cartella da organizzare (es. ./files):")
    else:
        folder_path = args.directory

    # controllo che la cartella esista
    if not os.path.isdir(folder_path):
        print(f"Errore: la cartella '{folder_path}' non esiste o non è accessibile.")
        sys.exit(1)

    filename = args.filename
    file_path = os.path.join(folder_path, args.filename)

    # verifica esistenza file
    if not os.path.isfile(file_path):
        print(f"Errore: il file '{args.filename}' non esiste nella cartella '{folder_path}'")
        return

    # ottengo dimensione file
    size = os.path.getsize(file_path)
        
    # ottengo il tipo MINE del file (es. 'image/jperg')
    mime_type, _ = mimetypes.guess_type(file_path)

    # classifica i file
    categoria = classifica_file(mime_type)


    
    
    if categoria is None:
        print(f"File '{args.filename}' di tipo '{mine_type}' non riconosciuto, nessuna azione eseguita.")
        return  

    # sposta i file nella relativa sottocartella
    nuovo_percorso = sposta_file(file_path, folder_path, categoria)

    
    # prepara le info per il file log in csv
    file_info = {
        'Nome': filename,
        'Tipo': mime_type,
        'Size Byte': size
    }
    
    # scrive nel file csv
    csv_path = os.path.join(folder_path, 'recap.csv')
    scrivi_log_csv(file_info, csv_path)

    print(f"Nome: {filename}")
    print(f"Tipo: {mime_type}")
    print(f"Dimensione: {size} byte")
    print(f"-> {categoria}/")
    print("-"*40)

if __name__=='__main__':
    main()